## 必要なライブラリのインポート  
Phi-4モデルを読み込み、使用するために必要なPyTorchとtransformersライブラリをインポートします。


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## ランダムシードの設定
ランダムシードを設定して、異なる実行間で結果を再現可能にします。


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoningモデルとトークナイザーの読み込み
MicrosoftのPhi-4-mini-flash-reasoningモデルと対応するトークナイザーをHugging Faceから読み込みます。モデルはCUDA上で読み込まれ、推論を高速化します。


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## 入力メッセージを準備する  
モデルのチャットテンプレートを使用して、二次方程式の数学問題を含む会話メッセージを作成します。


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## 応答を生成する
指定されたパラメーター（例えば、temperatureやtop_p）を使用して、出力のランダム性を制御しながら応答を生成します。


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## 出力をテキストにデコードする
生成されたトークンシーケンスを人間が読めるテキストに変換し、元の入力トークンを除外してモデルの応答のみを表示します。


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当方は一切の責任を負いません。
